In [2]:
import pandas as pd
import numpy as np
import pymc as pm
from sklearn.cluster import KMeans
import random
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture

# Data Processing for Clustering

In [3]:
flipped = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/Various%20Data%20Sets/data_flipped.csv")
flipped

/var/folders/d9/lqnstgr57q38yp_l9kjdv69c0000gn/T/ipykernel_58450/3024743246.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  flipped = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/Various%20Data%20Sets/data_flipped.csv")


,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y,chip_side,end_frame,Left_end,Right_end,Flip
0,202109090097,41.56,23.92,2021090900,97,53441.0,6,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.12,1.99,none,38,53441.0,42403.0,1
1,202109090097,41.56,23.92,2021090900,97,53441.0,7,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.06,1.96,none,38,53441.0,42403.0,1
2,202109090097,41.56,23.92,2021090900,97,53441.0,8,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.96,1.91,none,38,53441.0,42403.0,1
3,202109090097,41.56,23.92,2021090900,97,53441.0,9,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.80,1.85,none,38,53441.0,42403.0,1
4,202109090097,41.56,23.92,2021090900,97,53441.0,10,2021-09-10 00:26:32,11.0,DAL,...,TRADITIONAL,0,6.0,1.56,1.72,none,38,53441.0,42403.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173595,20211025003926,85.95,23.68,2021102500,3926,NaN,40,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.86,-2.19,none,44,37097.0,46083.0,0
1173596,20211025003926,85.95,23.68,2021102500,3926,NaN,41,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.94,-2.56,none,44,37097.0,46083.0,0
1173597,20211025003926,85.95,23.68,2021102500,3926,NaN,42,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.03,-2.95,none,44,37097.0,46083.0,0
1173598,20211025003926,85.95,23.68,2021102500,3926,NaN,43,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.12,-3.29,none,44,37097.0,46083.0,0


In [1158]:
frames_per_play = flipped[flipped['pff_positionLinedUp']=='QB'].groupby('uniqueplayId', as_index=False).size().rename(columns={'size':'play_length_frames'})

In [1159]:
flipped = flipped.merge(frames_per_play, on='uniqueplayId', how='left')

In [1160]:
flipped['play_length_frames'].min()

11.0

In [1161]:
flipped.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'gameId', 'playId', 'nflId',
       'frameId', 'time', 'jerseyNumber', 'team', 'playDirection', 'x', 'y',
       's', 'a', 'dis', 'o', 'dir', 'event', 'pff_role', 'pff_positionLinedUp',
       'pff_nflIdBlockedPlayer', 'pff_blockType', 'dropBackType',
       'pff_playAction', 'ball_snap_frame', 'new_x', 'new_y', 'chip_side',
       'end_frame', 'Left_end', 'Right_end', 'Flip', 'play_length_frames'],
      dtype='object')

In [1162]:
def name_func(pos):
    temp = {'nflId':pos+'_nflId',
     'jerseyNumber':pos+'_jerseyNumber',
     'team':pos+'_team', 
     'x':pos+'_x', 
     'y':pos+'_y',
     's':pos+'_s', 
     'a':pos+'_a', 
     'dis':pos+'_dis', 
     'o':pos+'_o', 
     'dir':pos+'_dir', 
     'pff_role':pos+'_pff_role', 
     'pff_positionLinedUp':pos+'_pff_positionLinedUp',
     'pff_nflIdBlockedPlayer':pos+'_pff_nflIdBlockedPlayer', 
     'pff_blockType':pos+'_pff_blockType', 
     'new_x':pos+'_new_x', 
     'new_y':pos+'_new_y', 
     'Flip':pos+'_Flip'}
    return temp

In [1163]:
qb = flipped[(flipped['pff_positionLinedUp']=='QB')]
qb = qb.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('QB')).reset_index(drop=True)

In [1164]:
qb_flip = flipped[(flipped['pff_positionLinedUp']=='QB_flip')]
qb_flip = qb_flip.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('QB')).reset_index(drop=True)

In [1165]:
ball = flipped[(flipped['pff_positionLinedUp']=='ball')]
ball = ball.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('ball')).reset_index(drop=True)

In [1166]:
ball_flip = flipped[(flipped['pff_positionLinedUp']=='ball_flip')]
ball_flip = ball_flip.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('ball')).reset_index(drop=True)


In [1167]:
rt = flipped[(flipped['pff_positionLinedUp']=='RT')]
rt = rt.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('Tackle')).reset_index(drop=True)

In [1168]:
lt = flipped[(flipped['pff_positionLinedUp']=='LT')]
lt = lt.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('Tackle')).reset_index(drop=True)

In [1169]:
le = flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==1)]
le = le.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('End')).reset_index(drop=True)

In [1170]:
re = flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==0)]
re = re.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('End')).reset_index(drop=True)

In [1171]:
play = flipped[(flipped['pff_positionLinedUp']=='QB')][['uniqueplayId', 'football_x', 'football_y', 
                                                        'gameId', 'playId', 'frameId', 'time', 'playDirection', 
                                                        'event', 'ball_snap_frame', 'end_frame', 'play_length_frames',
                                                        'Right_end', 'Left_end']].reset_index(drop=True)

In [1172]:
final_left = play.merge(qb, on=['uniqueplayId', 'frameId'], how='left').merge(ball, on=['uniqueplayId', 'frameId'], how='left')

In [1173]:
final_left = final_left.merge(lt, on=['uniqueplayId', 'frameId'], how='left').merge(re, on=['uniqueplayId', 'frameId'], how='left')

In [1174]:
final_right = play.merge(qb_flip, on=['uniqueplayId', 'frameId'], how='left').merge(ball_flip, on=['uniqueplayId', 'frameId'], how='left')

In [1175]:
final_right = final_right.merge(rt, on=['uniqueplayId', 'frameId'], how='left').merge(le, on=['uniqueplayId', 'frameId'], how='left')

In [1176]:
final = pd.concat([final_left, final_right])

In [1177]:
play_ex = final[final['uniqueplayId']==20210919023392]

Before classifying:<br>
> - Remove frames before ball is snapped and after it is thrown<br>
> - Remove tackle and end where the end's pff_role is Coverage<br>
> - First check: Remove plays where end moves into the inside (stunt)<br>
> - Remove plays where the distance between end and tackle is large directly after ball snap (not against each other)<br>
<br>

### Classifying
##### Clustering into two groups<br>
Predictor variables:<br>
> - Percentage that tackle/end are looking at each other <br>
> - Percentage that Euclidian distance is less than 0.5 (0.75)<br>
> - Total distance traveled of defensive end<br>
> - Max speed during rush<br>

### Create vars for clustering

In [1178]:
def dist(x_1, y_1, x_2, y_2):
    return np.sqrt(np.sum([(x_1-x_2)**2, (y_1-y_2)**2], axis=0))

In [1179]:
final['tackle_end_dist'] = dist(final['Tackle_new_x'], final['Tackle_new_y'], final['End_new_x'], final['End_new_y'])

In [1180]:
#angle_range = 20
def check_angle(x, y):
    diff = abs(abs(x-y)-180)
    #if diff <= angle_range:
        #return 1
    #else:
        #return 0
    return diff

In [1181]:
final['tackle_end_facing'] = final.apply(lambda row: check_angle(row['Tackle_o'], row['End_o']),  axis=1)

In [1182]:
final

,uniqueplayId,football_x,football_y,gameId,playId,frameId,time,playDirection,event,ball_snap_frame,...,End_dir,End_pff_role,End_pff_positionLinedUp,End_pff_nflIdBlockedPlayer,End_pff_blockType,End_new_x,End_new_y,End_Flip,tackle_end_dist,tackle_end_facing
0,202109090097,41.56,23.92,2021090900,97,6,2021-09-10 00:26:31,right,ball_snap,6.0,...,247.75,Pass Rush,ROLB,NaN,NaN,2.34,8.71,0.0,6.241354,46.42
1,202109090097,41.56,23.92,2021090900,97,7,2021-09-10 00:26:31,right,None,6.0,...,252.79,Pass Rush,ROLB,NaN,NaN,2.27,8.70,0.0,6.217025,46.23
2,202109090097,41.56,23.92,2021090900,97,8,2021-09-10 00:26:31,right,None,6.0,...,248.14,Pass Rush,ROLB,NaN,NaN,2.12,8.62,0.0,6.085433,45.79
3,202109090097,41.56,23.92,2021090900,97,9,2021-09-10 00:26:31,right,None,6.0,...,246.73,Pass Rush,ROLB,NaN,NaN,1.93,8.54,0.0,5.926635,51.35
4,202109090097,41.56,23.92,2021090900,97,10,2021-09-10 00:26:32,right,None,6.0,...,243.55,Pass Rush,ROLB,NaN,NaN,1.55,8.32,0.0,5.577966,57.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141940,20211025003926,85.95,23.68,2021102500,3926,40,2021-10-26 03:14:08,left,None,7.0,...,34.10,Pass Rush,LOLB,NaN,NaN,-4.37,2.97,1.0,1.082081,21.00
141941,20211025003926,85.95,23.68,2021102500,3926,41,2021-10-26 03:14:08,left,None,7.0,...,32.35,Pass Rush,LOLB,NaN,NaN,-4.47,3.11,1.0,1.126099,26.49
141942,20211025003926,85.95,23.68,2021102500,3926,42,2021-10-26 03:14:08,left,None,7.0,...,30.61,Pass Rush,LOLB,NaN,NaN,-4.56,3.27,1.0,1.192057,27.78
141943,20211025003926,85.95,23.68,2021102500,3926,43,2021-10-26 03:14:08,left,None,7.0,...,29.79,Pass Rush,LOLB,NaN,NaN,-4.66,3.43,1.0,1.236932,31.07


In [1183]:
final['end_too_far'] = (final['frameId']>final['ball_snap_frame']+10) &(final['tackle_end_dist']>4)

In [1211]:
def full_agg(grouped_df):
    return pd.DataFrame({
    'end_too_far': [1 if grouped_df['end_too_far'].sum()>=1 else 0],
    'Mean_tackle_end_dist': [grouped_df[grouped_df['frameId']>=grouped_df['ball_snap_frame']+10]['tackle_end_dist'].mean()],
    'Mean_tackle_end_facing': [grouped_df['tackle_end_facing'].mean()],
    'End_dist_travel': [grouped_df['End_dis'].sum()],
    'Mean_end_speed': [grouped_df['End_s'].mean()]
})

cluster_data = final.groupby(['uniqueplayId', 'QB_Flip'], group_keys=True).apply(full_agg).reset_index().drop(['level_2'], axis=1)

In [1212]:
cluster_data

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed
0,202109090097,0.0,0,1.319290,40.264848,12.83,3.519394
1,202109090097,1.0,0,0.910542,67.690606,9.78,2.924242
2,202109120163,0.0,0,1.068536,33.653636,6.99,3.195455
3,202109120163,1.0,0,0.934738,28.871364,5.88,2.608182
4,202109120288,0.0,0,1.566392,114.217778,4.36,2.177778
...,...,...,...,...,...,...,...
9647,20211025003684,1.0,0,1.162602,59.429111,12.31,2.685333
9648,20211025003904,0.0,0,0.670306,24.842308,7.83,2.950000
9649,20211025003904,1.0,0,0.907717,27.240769,8.22,3.070000
9650,20211025003926,0.0,0,0.747816,39.758947,6.23,1.620526


In [1202]:
# cluster_data = final.groupby(['uniqueplayId', 'QB_Flip'], as_index=False).agg({
#     'end_too_far': lambda x: 1 if x.sum()>=1 else 0,
#     'tackle_end_dist': 'mean',
#     'tackle_end_facing': 'mean',
#     'End_dis': 'sum',
#     'End_s':'mean'
# }).rename(columns={
#      'tackle_end_dist':'Mean_tackle_end_dist',
#      'tackle_end_facing':'Mean_tackle_end_facing',
#      'End_dis':'End_dist_travel',
#      'End_s':'Mean_end_speed'
#  })
# cluster_data

In [1215]:
power = final[(final['uniqueplayId']==2021091902389)&(final['QB_Flip']==0)]
speed = final[(final['uniqueplayId']==20210919023392)&(final['QB_Flip']==0)]
power_clust = cluster_data[(cluster_data['uniqueplayId']==2021091902389)&(cluster_data['QB_Flip']==0)]
speed_clust = cluster_data[(cluster_data['uniqueplayId']==20210919023392)&(cluster_data['QB_Flip']==0)]

In [1216]:
power_clust

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed
524,2021091902389,0.0,0,0.83702,14.549459,8.22,2.15973


In [1217]:
speed_clust

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed
3564,20210919023392,0.0,0,0.654655,63.854615,10.99,4.118077


In [1218]:
cluster_final = cluster_data[cluster_data['end_too_far']==0]

# K-Means Clustering

In [1257]:
test_plays = [20210926112818,2021092602494,20211010112817,20211017071173,2021101010874,20211017024110, 20211003081723, 2021091901500, 20211021002175, 20210926051887]

cluster_final_test = cluster_final[cluster_final['uniqueplayId'].isin(test_plays)]

cluster_final_train = cluster_final[~cluster_final['uniqueplayId'].isin(test_plays)]

cluster_X = cluster_final_train.drop(
    ['uniqueplayId', 'QB_Flip', 'end_too_far'], axis=1)

scaler = StandardScaler().fit(cluster_X)
cluster_X_scale_train = scaler.transform(cluster_X)
cluster_X_scale_test = scaler.transform(cluster_final_test.drop(
    ['uniqueplayId', 'QB_Flip', 'end_too_far'], axis=1))

In [1258]:
kmeans = KMeans(n_clusters=2, random_state=2023).fit(cluster_X_scale_train)

In [1259]:
cluster_final_test['Cluster'] = kmeans.predict(cluster_X_scale_test)
cluster_final_test['actual'] = [1,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,0,1,1,1]

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/2183708035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_final_test['Cluster'] = kmeans.predict(cluster_X_scale_test)
/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/2183708035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_final_test['actual'] = [1,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,0,1,1,1]


In [1260]:
cluster_final_test

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,Cluster,actual
506,2021091901500,0.0,0,1.043156,11.141579,3.14,1.608421,1,1
507,2021091901500,1.0,0,1.409465,25.605263,6.98,3.710526,0,0
803,2021092602494,0.0,0,1.018627,16.781563,7.98,2.393750,1,1
804,2021092602494,1.0,0,1.114277,68.421250,8.79,2.676250,0,0
1589,2021101010874,0.0,0,0.992189,26.089355,9.05,2.798710,0,1
1590,2021101010874,1.0,0,0.919114,68.245806,10.83,3.374516,0,0
4802,20210926051887,0.0,0,1.973584,37.672353,5.20,2.971765,1,1
4803,20210926051887,1.0,0,0.867127,23.635882,4.69,2.740000,1,1
5317,20210926112818,0.0,0,0.776541,67.188125,10.49,3.303125,0,0
5318,20210926112818,1.0,0,1.537078,31.973437,9.29,2.905937,0,1


In [1261]:
# Accuracy
sum(cluster_final_test['actual']==cluster_final_test['Cluster'])/len(cluster_final_test)

0.75

In [1262]:
labs = kmeans.labels_
cluster_final_train['rushType'] = labs

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/2913040956.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_final_train['rushType'] = labs


In [1263]:
cluster_final_train[cluster_final_train['uniqueplayId']==20211017063573]

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,rushType
8249,20211017063573,0.0,0,1.433756,16.875161,10.47,3.397097,0
8250,20211017063573,1.0,0,1.031505,61.495161,13.88,4.518710,0


In [1264]:
kmeans.cluster_centers_

array([[-0.05996801,  0.37507738,  0.78750591,  0.74533959],
       [ 0.04752204, -0.2972325 , -0.62406416, -0.59064919]])

# Gaussian Mixture Model

In [1265]:
gm = GaussianMixture(n_components=2, random_state=6030).fit(cluster_X)

In [1266]:
gmm_preds = gm.predict(cluster_final_test.drop(
    ['uniqueplayId', 'QB_Flip', 'end_too_far', 'actual', 'Cluster'], axis=1))
gmm_preds = [(i-1)*-1 for i in gmm_preds]
cluster_final_test['GMM_Pred'] = kmeans.predict(cluster_X_scale_test)

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/2331119997.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_final_test['GMM_Pred'] = kmeans.predict(cluster_X_scale_test)


In [1267]:
cluster_final_test

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,Cluster,actual,GMM_Pred
506,2021091901500,0.0,0,1.043156,11.141579,3.14,1.608421,1,1,1
507,2021091901500,1.0,0,1.409465,25.605263,6.98,3.710526,0,0,0
803,2021092602494,0.0,0,1.018627,16.781563,7.98,2.393750,1,1,1
804,2021092602494,1.0,0,1.114277,68.421250,8.79,2.676250,0,0,0
1589,2021101010874,0.0,0,0.992189,26.089355,9.05,2.798710,0,1,0
1590,2021101010874,1.0,0,0.919114,68.245806,10.83,3.374516,0,0,0
4802,20210926051887,0.0,0,1.973584,37.672353,5.20,2.971765,1,1,1
4803,20210926051887,1.0,0,0.867127,23.635882,4.69,2.740000,1,1,1
5317,20210926112818,0.0,0,0.776541,67.188125,10.49,3.303125,0,0,0
5318,20210926112818,1.0,0,1.537078,31.973437,9.29,2.905937,0,1,0


In [1268]:
# Accuracy
sum(cluster_final_test['actual']==cluster_final_test['GMM_Pred'])/len(cluster_final_test)

0.75

In [1269]:
gm.means_

array([[ 1.16615885, 26.1976727 ,  7.27401843,  2.78494659],
       [ 1.15544887, 61.32940427,  9.67472515,  2.98266389]])

In [1270]:
gmm_pred_prob = gm.predict_proba(cluster_final_test.drop(
    ['uniqueplayId', 'QB_Flip', 'end_too_far', 'actual', 'Cluster', 'GMM_Pred'], axis=1))
gmm_pred_prob = pd.DataFrame(gmm_pred_prob).round(5)
gmm_pred_prob.columns = ['Speed_prob', 'Power_prob']

In [1271]:
pd.concat([cluster_final_test, gmm_pred_prob.set_index(cluster_final_test.index)], axis=1)

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,Cluster,actual,GMM_Pred,Speed_prob,Power_prob
506,2021091901500,0.0,0,1.043156,11.141579,3.14,1.608421,1,1,1,0.97700,0.02300
507,2021091901500,1.0,0,1.409465,25.605263,6.98,3.710526,0,0,0,0.98435,0.01565
803,2021092602494,0.0,0,1.018627,16.781563,7.98,2.393750,1,1,1,0.96016,0.03984
804,2021092602494,1.0,0,1.114277,68.421250,8.79,2.676250,0,0,0,0.01848,0.98152
1589,2021101010874,0.0,0,0.992189,26.089355,9.05,2.798710,0,1,0,0.94629,0.05371
1590,2021101010874,1.0,0,0.919114,68.245806,10.83,3.374516,0,0,0,0.01431,0.98569
4802,20210926051887,0.0,0,1.973584,37.672353,5.20,2.971765,1,1,1,0.98245,0.01755
4803,20210926051887,1.0,0,0.867127,23.635882,4.69,2.740000,1,1,1,0.96593,0.03407
5317,20210926112818,0.0,0,0.776541,67.188125,10.49,3.303125,0,0,0,0.02428,0.97572
5318,20210926112818,1.0,0,1.537078,31.973437,9.29,2.905937,0,1,0,0.91661,0.08339


In [1272]:
cluster_final_train['GMM_rushType'] = (gm.predict(cluster_X)-1)*-1
gmm_pred_prob_train = gm.predict_proba(cluster_final_train.drop(
    ['uniqueplayId', 'QB_Flip', 'end_too_far', 'rushType', 'GMM_rushType'], axis=1))
gmm_pred_prob_train = pd.DataFrame(gmm_pred_prob_train).round(5)
gmm_pred_prob_train.columns = ['Power_prob', 'Speed_prob']
cluster_final_train = pd.concat([cluster_final_train, gmm_pred_prob_train.set_index(cluster_final_train.index)], axis=1)

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/592465473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_final_train['GMM_rushType'] = (gm.predict(cluster_X)-1)*-1


In [1273]:
cluster_final_train

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,rushType,GMM_rushType,Power_prob,Speed_prob
0,202109090097,0.0,0,1.319290,40.264848,12.83,3.519394,0,0,0.18422,0.81578
1,202109090097,1.0,0,0.910542,67.690606,9.78,2.924242,0,0,0.02026,0.97974
2,202109120163,0.0,0,1.068536,33.653636,6.99,3.195455,1,1,0.96468,0.03532
3,202109120163,1.0,0,0.934738,28.871364,5.88,2.608182,1,1,0.97444,0.02556
4,202109120288,0.0,0,1.566392,114.217778,4.36,2.177778,1,0,0.00000,1.00000
...,...,...,...,...,...,...,...,...,...,...,...
9647,20211025003684,1.0,0,1.162602,59.429111,12.31,2.685333,0,0,0.00353,0.99647
9648,20211025003904,0.0,0,0.670306,24.842308,7.83,2.950000,1,1,0.97425,0.02575
9649,20211025003904,1.0,0,0.907717,27.240769,8.22,3.070000,0,1,0.97167,0.02833
9650,20211025003926,0.0,0,0.747816,39.758947,6.23,1.620526,1,1,0.88360,0.11640


In [1274]:
sum(cluster_final_train['GMM_rushType']!=cluster_final_train['rushType'])

3015

In [1275]:
cluster_filt = cluster_final_train[(cluster_final_train['Power_prob']>.55)|(cluster_final_train['Power_prob']<.45)]

In [1276]:
cluster_filt

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,rushType,GMM_rushType,Power_prob,Speed_prob
0,202109090097,0.0,0,1.319290,40.264848,12.83,3.519394,0,0,0.18422,0.81578
1,202109090097,1.0,0,0.910542,67.690606,9.78,2.924242,0,0,0.02026,0.97974
2,202109120163,0.0,0,1.068536,33.653636,6.99,3.195455,1,1,0.96468,0.03532
3,202109120163,1.0,0,0.934738,28.871364,5.88,2.608182,1,1,0.97444,0.02556
4,202109120288,0.0,0,1.566392,114.217778,4.36,2.177778,1,0,0.00000,1.00000
...,...,...,...,...,...,...,...,...,...,...,...
9647,20211025003684,1.0,0,1.162602,59.429111,12.31,2.685333,0,0,0.00353,0.99647
9648,20211025003904,0.0,0,0.670306,24.842308,7.83,2.950000,1,1,0.97425,0.02575
9649,20211025003904,1.0,0,0.907717,27.240769,8.22,3.070000,0,1,0.97167,0.02833
9650,20211025003926,0.0,0,0.747816,39.758947,6.23,1.620526,1,1,0.88360,0.11640


In [1277]:
cluster_filt = cluster_filt.drop('rushType', axis=1).rename(columns={'GMM_rushType':'rushType'})

In [1278]:
cluster_filt['rushType'] = cluster_filt['rushType'].map({0:'Speed', 1:'Power'})

In [1279]:
cluster_filt

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,rushType,Power_prob,Speed_prob
0,202109090097,0.0,0,1.319290,40.264848,12.83,3.519394,Speed,0.18422,0.81578
1,202109090097,1.0,0,0.910542,67.690606,9.78,2.924242,Speed,0.02026,0.97974
2,202109120163,0.0,0,1.068536,33.653636,6.99,3.195455,Power,0.96468,0.03532
3,202109120163,1.0,0,0.934738,28.871364,5.88,2.608182,Power,0.97444,0.02556
4,202109120288,0.0,0,1.566392,114.217778,4.36,2.177778,Speed,0.00000,1.00000
...,...,...,...,...,...,...,...,...,...,...
9647,20211025003684,1.0,0,1.162602,59.429111,12.31,2.685333,Speed,0.00353,0.99647
9648,20211025003904,0.0,0,0.670306,24.842308,7.83,2.950000,Power,0.97425,0.02575
9649,20211025003904,1.0,0,0.907717,27.240769,8.22,3.070000,Power,0.97167,0.02833
9650,20211025003926,0.0,0,0.747816,39.758947,6.23,1.620526,Power,0.88360,0.11640


In [1280]:
cluster_final_test = cluster_final_test.drop(['Cluster', 'GMM_Pred'], axis=1).rename(columns={'actual':'rushType'})
cluster_final_test['rushType'] = cluster_final_test['rushType'].map({0:'Speed', 1:'Power'})

In [1281]:
cluster_final_test

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,rushType
506,2021091901500,0.0,0,1.043156,11.141579,3.14,1.608421,Power
507,2021091901500,1.0,0,1.409465,25.605263,6.98,3.710526,Speed
803,2021092602494,0.0,0,1.018627,16.781563,7.98,2.393750,Power
804,2021092602494,1.0,0,1.114277,68.421250,8.79,2.676250,Speed
1589,2021101010874,0.0,0,0.992189,26.089355,9.05,2.798710,Power
1590,2021101010874,1.0,0,0.919114,68.245806,10.83,3.374516,Speed
4802,20210926051887,0.0,0,1.973584,37.672353,5.20,2.971765,Power
4803,20210926051887,1.0,0,0.867127,23.635882,4.69,2.740000,Power
5317,20210926112818,0.0,0,0.776541,67.188125,10.49,3.303125,Speed
5318,20210926112818,1.0,0,1.537078,31.973437,9.29,2.905937,Power


In [1282]:
cluster_filt = pd.concat([cluster_filt, cluster_final_test], axis=0)

In [1283]:
cluster_filt

,uniqueplayId,QB_Flip,end_too_far,Mean_tackle_end_dist,Mean_tackle_end_facing,End_dist_travel,Mean_end_speed,rushType,Power_prob,Speed_prob
0,202109090097,0.0,0,1.319290,40.264848,12.83,3.519394,Speed,0.18422,0.81578
1,202109090097,1.0,0,0.910542,67.690606,9.78,2.924242,Speed,0.02026,0.97974
2,202109120163,0.0,0,1.068536,33.653636,6.99,3.195455,Power,0.96468,0.03532
3,202109120163,1.0,0,0.934738,28.871364,5.88,2.608182,Power,0.97444,0.02556
4,202109120288,0.0,0,1.566392,114.217778,4.36,2.177778,Speed,0.00000,1.00000
...,...,...,...,...,...,...,...,...,...,...
7985,20211017024110,1.0,0,0.766184,24.512857,8.35,2.832857,Power,NaN,NaN
8279,20211017071173,0.0,0,1.820526,12.505000,5.57,2.818000,Speed,NaN,NaN
8280,20211017071173,1.0,0,1.207437,17.531500,2.84,1.356000,Power,NaN,NaN
8740,20211021002175,0.0,0,1.485747,16.966111,4.61,2.515556,Power,NaN,NaN


In [1284]:
cluster_filt = cluster_filt[['uniqueplayId', 'QB_Flip', 'rushType', 'Power_prob', 'Speed_prob']].merge(
    final.drop('end_too_far', axis=1), how='left', on=['uniqueplayId', 'QB_Flip']
)

In [1285]:
# cluster_filt.to_csv('clustered.csv', index = False)